# Test 3, Question 37
> **Hint:** In Databricks, import code for all questions via this URL:
> 
> https://github.com/flrs/spark_practice_tests_code/raw/main/spark_practice_tests_code.dbc

In [0]:
%run ./create_transactionsDf

In [0]:
transactionsDf = transactionsDf.repartition(1)
transactionsDf.printSchema()
transactionsDf.show()

root
-- transactionId: integer (nullable = true)
-- predError: integer (nullable = true)
-- value: integer (nullable = true)
-- storeId: integer (nullable = true)
-- productId: integer (nullable = true)
-- f: integer (nullable = true)

+-------------+---------+-----+-------+---------+----+
transactionId|predError|value|storeId|productId| f|
+-------------+---------+-----+-------+---------+----+
 1| 3| 4| 25| 1|null|
 2| 6| 7| 2| 2|null|
 3| 3| null| 25| 3|null|
 4| null| null| 3| 2|null|
 5| null| null| null| 2|null|
 6| 3| 2| 25| 2|null|
+-------------+---------+-----+-------+---------+----+

In [0]:
from pyspark.sql.functions import lit

contractsDf = transactionsDf.withColumn('tax_id', lit(999)).withColumnRenamed('productId', 'rollId')
contractsDf.printSchema()
contractsDf.show()

root
-- transactionId: integer (nullable = true)
-- predError: integer (nullable = true)
-- value: integer (nullable = true)
-- storeId: integer (nullable = true)
-- rollId: integer (nullable = true)
-- f: integer (nullable = true)
-- tax_id: integer (nullable = false)

+-------------+---------+-----+-------+------+----+------+
transactionId|predError|value|storeId|rollId| f|tax_id|
+-------------+---------+-----+-------+------+----+------+
 1| 3| 4| 25| 1|null| 999|
 2| 6| 7| 2| 2|null| 999|
 3| 3| null| 25| 3|null| 999|
 4| null| null| 3| 2|null| 999|
 5| null| null| null| 2|null| 999|
 6| 3| 2| 25| 2|null| 999|
+-------------+---------+-----+-------+------+----+------+

In [0]:
filePath = '/FileStore/data_157.parquet'

transactionsDf.write.mode('overwrite').parquet(filePath)
contractsDf.write.mode('append').parquet(filePath)

## Answer 1 (incorrect)

In [0]:
spark.read.parquet(filePath, mergeSchema='y').show()

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-2042601433084332> in <module> 
 ----> 1 spark . read . parquet ( filePath , mergeSchema = 'y' ) . show ( ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 IllegalArgumentException : For input string: "y"

## Answer 2 (correct)

In [0]:
spark.read.option("mergeSchema", "true").parquet(filePath).show()

+-------------+---------+-----+-------+---------+----+------+------+
transactionId|predError|value|storeId|productId| f|rollId|tax_id|
+-------------+---------+-----+-------+---------+----+------+------+
 1| 3| 4| 25| null|null| 1| 999|
 2| 6| 7| 2| null|null| 2| 999|
 3| 3| null| 25| null|null| 3| 999|
 4| null| null| 3| null|null| 2| 999|
 5| null| null| null| null|null| 2| 999|
 6| 3| 2| 25| null|null| 2| 999|
 1| 3| 4| 25| 1|null| null| null|
 2| 6| 7| 2| 2|null| null| null|
 3| 3| null| 25| 3|null| null| null|
 4| null| null| 3| 2|null| null| null|
 5| null| null| null| 2|null| null| null|
 6| 3| 2| 25| 2|null| null| null|
+-------------+---------+-----+-------+---------+----+------+------+

## Answer 3 (incorrect)

In [0]:
spark.read.parquet(filePath).show()

+-------------+---------+-----+-------+---------+----+
transactionId|predError|value|storeId|productId| f|
+-------------+---------+-----+-------+---------+----+
 1| 3| 4| 25| null|null|
 2| 6| 7| 2| null|null|
 3| 3| null| 25| null|null|
 4| null| null| 3| null|null|
 5| null| null| null| null|null|
 6| 3| 2| 25| null|null|
 1| 3| 4| 25| 1|null|
 2| 6| 7| 2| 2|null|
 3| 3| null| 25| 3|null|
 4| null| null| 3| 2|null|
 5| null| null| null| 2|null|
 6| 3| 2| 25| 2|null|
+-------------+---------+-----+-------+---------+----+

## Answer 4 (incorrect)

In [0]:
nx = 0
for file in dbutils.fs.ls(filePath):
  if not file.name.endswith('.parquet'):
    continue
  df_temp = spark.read.parquet(file.path)
  if nx == 0:
    df = df_temp
  else:
    df = df.union(df_temp)
  nx = nx+1
df.show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2042601433084330> in <module> 
 10 df = df_temp
 11 else : 
 ---> 12 df = df . union ( df_temp ) 
 13 nx = nx + 1 
 14 df . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in union (self, other) 
 1849 Also as standard in SQL , this function resolves columns by position ( not by name ) . 
 1850 """
 -> 1851 return DataFrame ( self . _jdf . union ( other . _jdf ) , self . sql_ctx ) 
 1852 
 1853 @ since ( 1.3 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Union can only be performed on tables with the same number of columns, but the first table has 6 columns and the second table has 7 columns;
'Union false, false
:- Relation[transactionId#345,predError#346,value#347,storeId#348,productId#349,f#350] parquet
+- Relation[transactionId#357,predError#358,value#359,storeId#360,rollId#361,f#362,tax_id#363] parquet

## Answer 5 (incorrect)

In [0]:
nx = 0
for file in dbutils.fs.ls(filePath):
  if not file.name.endswith('.parquet'):
    continue
  df_temp = spark.read.parquet(file.path)
  if nx == 0:
    df = df_temp
  else:
    df = df.join(df_temp, how='outer')
  nx = nx+1
df.show()

+-------------+---------+-----+-------+------+----+------+-------------+---------+-----+-------+---------+----+
transactionId|predError|value|storeId|rollId| f|tax_id|transactionId|predError|value|storeId|productId| f|
+-------------+---------+-----+-------+------+----+------+-------------+---------+-----+-------+---------+----+
 1| 3| 4| 25| 1|null| 999| 1| 3| 4| 25| 1|null|
 1| 3| 4| 25| 1|null| 999| 2| 6| 7| 2| 2|null|
 1| 3| 4| 25| 1|null| 999| 3| 3| null| 25| 3|null|
 1| 3| 4| 25| 1|null| 999| 4| null| null| 3| 2|null|
 1| 3| 4| 25| 1|null| 999| 5| null| null| null| 2|null|
 1| 3| 4| 25| 1|null| 999| 6| 3| 2| 25| 2|null|
 2| 6| 7| 2| 2|null| 999| 1| 3| 4| 25| 1|null|
 2| 6| 7| 2| 2|null| 999| 2| 6| 7| 2| 2|null|
 2| 6| 7| 2| 2|null| 999| 3| 3| null| 25| 3|null|
 2| 6| 7| 2| 2|null| 999| 4| null| null| 3| 2|null|
 2| 6| 7| 2| 2|null| 999| 5| null| null| null| 2|null|
 2| 6| 7| 2| 2|null| 999| 6| 3| 2| 25| 2|null|
 3| 3| null| 25| 3|null| 999| 1| 3| 4| 25| 1|null|
 3| 3| null| 25| 3|null| 999| 2| 6| 7| 2| 2|null|
 3| 3| null| 25| 3|null| 999| 3| 3| null| 25| 3|null|
 3| 3| null| 25| 3|null| 999| 4| null| null| 3| 2|null|
 3| 3| null| 25| 3|null| 999| 5| null| null| null| 2|null|
 3| 3| null| 25| 3|null| 999| 6| 3| 2| 25| 2|null|
 4| null| null| 3| 2|null| 999| 1| 3| 4| 25| 1|null|
 4| null| null| 3| 2|null| 999| 2| 6| 7| 2| 2|null|
+-------------+---------+-----+-------+------+----+------+-------------+---------+-----+-------+---------+----+
only showing top 20 rows